## Data

In [70]:
from cot import Collection
coll = Collection("worldtree", load_pregenerated_cots=True,generate_mode="recache") #does not load pre-generated cots

Loading worldtree...


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset worldtree_dataset downloaded and prepared to /Users/robertpraas/.cache/huggingface/datasets/worldtree_dataset/thoughtsource/1.0.0/4ec0cd827b41f05891af9a27bf461fecd407e2fe7c1beebfed1eb00193c2cd52. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [71]:
coll = coll.select(split="test", number_samples=2)
#coll.select_generated_cots(cot_trigger = "kojima-01")
#coll.select_generated_cots(api_service='cohere')

In [52]:
coll

| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |      2 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'med_qa_open', 'medmc_qa', '_init_', 'mmlu_clinical_knowledge', 'mmlu_college_biology', 'mmlu_college_medicine', 'mmlu_medical_genetics', 'mmlu_professional_medicine', '_init_', 'mmlu_anatomy', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [33]:
coll['worldtree']['test'][0]

{'id': 'worldtree_test_0',
 'ref_id': '',
 'question': 'What remains in the same location in the sky of the Northern Hemisphere each night?',
 'type': 'multiplechoice',
 'choices': ['the Sun', 'the Little Dipper', 'the North Star', 'the Moon'],
 'context': '',
 'cot': ['The north star does not move every night in the sky in the northern hemisphere.',
  'Moving changes position.',
  'To remain means to not change.',
  'If something does not move then that something can be found in the same location.',
  'Place is synonymous with position.'],
 'answer': ['the North Star'],
 'generated_cot': [],
 'feedback': []}

## Generate and extract

In [34]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI


In [35]:
"""CoT Chain"""

llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo")

template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [36]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")


In [37]:
"""CoT-Ans_extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [72]:
"""Generate CoT with use of TS-schema"""
#compare with config used before; what about max tokens?
#config contains what the chain needs
input_dict = {'input_dict':
              {
                  'chain': overall_chain,
                  "instruction": "Be faithful and a little hopeful",
                  "cot_trigger": "Answer: Let's think step by step.",
                  "answer_extraction": "Therefore, among A through D, the answer is",
                  'model': "gpt-3.5-turbo",
                  'temperature': 0,
                  'max_tokens': 800
              }
              }

coll.generate_extract_flexible(input_dict=input_dict)





Generating worldtree...


  0%|          | 0/2 [00:00<?, ?ex/s]



> Entering new SequentialChain chain...

> Finished chain.
{'id': 'worldtree_test_788', 'ref_id': '', 'question': 'When light hits a mirror, most of the light is', 'type': 'multiplechoice', 'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'], 'context': '', 'cot': ['A mirror reflects light.', 'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.', 'Light is a kind of wave.', 'When light hits a reflective object , that light bounces off that object.', 'A mirror is a kind of reflective object.'], 'answer': ['reflected.'], 'generated_cot': [{'id': '17eb19ff-79ba-4b6a-abe8-31b6b854be75', 'fragments_version': None, 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection. Therefore, the correct answe

In [56]:
coll['worldtree']['test'][0]['generated_cot'][0]

{'id': '7a42d609-d5c7-4272-865a-6822b9ed267f',
 'fragments_version': None,
 'instruction': 'Be faithful and a little hopeful',
 'cot_trigger': "Answer: Let's think step by step.",
 'cot_trigger_template': '',
 'prompt_text': '',
 'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection. Therefore, the correct answer is B) reflected.',
 'answers': [{'id': '14b14f65-4326-47d3-a91c-fbb3b84a9ee2',
   'answer_extraction': 'Therefore, among A through D, the answer is',
   'answer_extraction_template': '',
   'answer_extraction_text': '',
   'answer': 'B) reflected.',
   'answer_from_choices': '',
   'correct_answer': None}],
 'author': '',
 'date': '2023/06/12 11:47:59',
 'api_service': '',
 'model': "{'name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 800}",
 'comment': 'generated and extracted',
 'annotations': []}

In [59]:
# Title for CoT trigger and extraction used
coll.evaluate(title='hopeful_Kojima')

  0%|          | 0/2 [00:00<?, ?ex/s]

{'worldtree': {'test': {'accuracy': {'gpt-3.5-turbo': {'hopeful_Kojima': 1.0}}}}}

## Generate or extract

In [60]:
#from langchain.llms.openai import OpenAIChat

from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json

In [64]:
"""Extract script: Assumes there are CoTs in the dataset already"""

input_dict = {
    'input_dict': {
        'chain': answer_chain,
        "instruction": "Be faithful and a little hopeful",
        "cot_trigger": "Answer: Let's think step by step.",
        "answer_extraction": "Therefore, among A through D, the answer is",
        'model': "gpt-3.5-turbo",
        'api_service': 'OpenAI',
        'temperature': 0,
        'max_tokens': 800
    }
} 
coll.extract_flexible(input_dict=input_dict)

Generating worldtree...


  0%|          | 0/2 [00:00<?, ?ex/s]

[{'id': '14b14f65-4326-47d3-a91c-fbb3b84a9ee2', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'B) reflected.', 'answer_from_choices': 'B', 'correct_answer': True}]
################
[{'id': '14b14f65-4326-47d3-a91c-fbb3b84a9ee2', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'B) reflected.', 'answer_from_choices': 'B', 'correct_answer': True}, {'id': '900c7cda-1d07-4092-9d6c-48226fce1cab', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'B) reflected.', 'answer_from_choices': '', 'correct_answer': None}]
[{'id': '8ac394a9-59d6-4433-935a-4d6b8d84671a', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'B) flammabilit

In [66]:
"""Generate only"""
input_dict = {
    'input_dict': {
        'chain': cot_chain,
        "instruction": "Be faithful and a little hopeful",
        "cot_trigger": "Answer: Let's think step by step.",
        "answer_extraction": "Therefore, among A through D, the answer is",
        'model': "gpt-3.5-turbo",
        'api_service': 'OpenAI',
        'temperature': 0,
        'max_tokens': 800
    }
} 
coll.generate_flexible(input_dict=input_dict)

Generating worldtree...


  0%|          | 0/2 [00:00<?, ?ex/s]

In [ ]:
coll['worldtree']['test'][0]['generated_cot'][2]

## Reflection

In [68]:
llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo")

reflect_template = """
    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}
    Answer: {answer}
    
    {reflection_prompt}
    """
prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt'], template=reflect_template)
reflect_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection")

extraction_template = """{instruction}

    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}{answer}
    {reflection_prompt}{reflection}

    {reflect_answer_extraction}
    """
    #Get reflection
prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt','reflection','reflect_answer_extraction'], template=extraction_template)
reflect_answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection_answer")

    # This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
reflect_overall_chain = SequentialChain(chains=[reflect_chain, reflect_answer_chain],input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction",'cot','answer','reflection_prompt','reflect_answer_extraction'],
        output_variables=["reflection", "reflection_answer"],
        verbose=True)




In [73]:
#check for what is already in item
input_dict = {'input_dict':{
    'chain':reflect_overall_chain,
    'cot_trigger':"", 
    'answer':"", 
    'answer_extraction': "", 
    'cot': "", 
    'instruction': "",
    'api_service': "chat_openai", 
    'model': "gpt-3.5-turbo",
    'reflection_prompt':"Double check this",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',
    'temperature': 0,
    'max_tokens': 800 
}
}
coll.metareason_flexible(input_dict=input_dict)

Generating worldtree...


  0%|          | 0/2 [00:00<?, ?ex/s]



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


In [74]:
coll['worldtree']['test'][0]['generated_cot'][1]

{'id': '530e51f1-39d2-4fa5-a586-efba3dfa753e',
 'fragments_version': '',
 'instruction': '',
 'cot_trigger': 'Double check this',
 'cot_trigger_template': '',
 'prompt_text': '',
 'cot': 'As an AI language model, I can confirm that the answer is correct. When light hits a mirror, it is reflected back off the surface of the mirror. This is due to the smooth surface of the mirror, which allows the light to bounce back in a predictable way. Therefore, the correct answer is B) reflected.',
 'answers': [{'id': 'acae3361-0db6-44c9-a51d-cabe875c9a33',
   'answer_extraction': 'Based on the reflection, what is the definite answer?',
   'answer_extraction_template': '',
   'answer_extraction_text': 'self_reflection',
   'answer': 'The definite answer is B) reflected.',
   'answer_from_choices': '',
   'correct_answer': None}],
 'author': '',
 'date': '2023/06/12 12:06:42',
 'api_service': 'chat_openai',
 'model': "{'name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 800}",
 'comment': 'self